# Imports

In [43]:
import numpy as np
import pandas as pd 

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

import re
from datetime import datetime

# Loading Data

In [44]:
train_path = "data/train.csv"
test_path = "data/test_x.csv"

train_raw = pd.read_csv(train_path)
test_raw = pd.read_csv(test_path)

train_y_raw = train_raw['Degerlendirme Puani']
train_X_raw = train_raw.drop('Degerlendirme Puani', axis=1)

test_X_raw = test_raw

/tmp/ipykernel_1795/3918458119.py:4: DtypeWarning: Columns (13,15,19,30,32,37,41,42) have mixed types. Specify dtype option on import or set low_memory=False.
  train_raw = pd.read_csv(train_path)


# Preprocessing

## Features

### Doğum tarihi ve Yaş

In [101]:
def tarih_formatla(df):
    def extract_year(date_str):
        if not isinstance(date_str, str):
            return None
        
        # Farklı tarih formatları için regex pattern'leri
        patterns = [
            r'(\d{4})[-/.](\d{1,2})[-/.](\d{1,2})',  # YYYY-MM-DD, YYYY/MM/DD, YYYY.MM.DD
            r'(\d{1,2})[-/.](\d{1,2})[-/.](\d{4})',  # DD-MM-YYYY, DD/MM/YYYY, DD.MM.YYYY
            r'(\d{2})[-/.](\d{1,2})[-/.](\d{1,2})',  # YY-MM-DD, YY/MM/DD, YY.MM.DD
            r'(\d{1,2})[-/.](\d{1,2})[-/.](\d{2})'   # DD-MM-YY, DD/MM/YY, DD.MM.YY
        ]
        
        for pattern in patterns:
            match = re.search(pattern, date_str)
            if match:
                groups = match.groups()
                if len(groups[0]) == 4:  # YYYY MM DD formatı
                    year = int(groups[0])
                elif len(groups[2]) == 4:  # DD MM YYYY formatı
                    year = int(groups[2])
                else:  # YY-MM-DD veya DD-MM-YY formatı
                    year = int(groups[0] if len(groups[0]) == 2 else groups[2])
                    year += 1900 if year > 24 else 2000
                return year
        
        return np.nan
    
    def yas_sinirla(entry):
        dogum_tarihi = entry['Dogum Tarihi']
        basvurudugu_yil = entry['Basvuru Yili']
        return dogum_tarihi if 1900 <= dogum_tarihi <= basvurudugu_yil - 18 else np.nan
    
    # Yılları çıkar
    df['Dogum Tarihi'] = df["Dogum Tarihi"].apply(extract_year)
    df['Dogum Tarihi'] = df['Dogum Tarihi'].apply(lambda x: x if 1900 <= x <= 2007 else np.nan)
    df['yas'] = 2024 - df['Dogum Tarihi']
    # Geçersiz yılları ortalama ile doldur
    mean_year = df['yas'].mean()
    df['yas'] = df['yas'].fillna(round(mean_year)).astype(int)
    
    
    return df

train_X_raw['Dogum Tarihi'] = train_X_raw['Dogum Tarihi'].replace('1/1/70 2:00', np.nan)
# Başvuru yılı ve Doğum tarihi feature ı kullanarak yaş feature ı üretildi.
train_X_raw = tarih_formatla(train_X_raw)
train_X_raw = train_X_raw.drop("Dogum Tarihi", axis=1)
train_X_raw["yas"].value_counts()

yas
26    20516
24     6341
25     5941
27     5704
23     5089
28     4983
29     4343
30     3283
22     2966
31     2383
32     1476
33      846
21      737
34      240
35       62
54       58
36       34
37       25
20       24
38       18
19       10
40        6
17        5
39        5
99        4
41        4
95        4
48        3
18        2
97        2
44        2
47        2
45        2
42        1
52        1
91        1
80        1
74        1
Name: count, dtype: int64

### Cinsiyet(One Hot)

In [102]:
train_X_raw["Cinsiyet"] = train_X_raw["Cinsiyet"].str.lower()
train_X_raw["Cinsiyet"] = train_X_raw["Cinsiyet"].replace(to_replace=[np.nan, "belirtmek istemiyorum"], value="diger")

#train_X_raw["Cinsiyet"].value_counts(dropna=False)

Cinsiyet
erkek    32764
kadın    32077
diger      284
Name: count, dtype: int64

### Doğum Yeri(Frequency) ve İkametgah Şehri(One Hot)

In [27]:
from rapidfuzz import process

sehirler=["Adana", "Adıyaman", "Afyon", "Ağrı", "Amasya", "Ankara", "Antalya", "Artvin", "Aydın", "Balıkesir", "Bilecik", "Bingöl", "Bitlis", "Bolu", "Burdur", "Bursa", "Çanakkale", "Çankırı", "Çorum", "Denizli", "Diyarbakır", "Edirne", "Elazığ", "Erzincan", "Erzurum", "Eskişehir", "Gaziantep", "Giresun", "Gümüşhane", "Hakkari", "Hatay", "Isparta", "İçel (Mersin)", "İstanbul", "İzmir", "Kars", "Kastamonu", "Kayseri", "Kırklareli", "Kırşehir", "Kocaeli", "Konya", "Kütahya", "Malatya", "Manisa", "Kahramanmaraş", "Mardin", "Muğla", "Muş", "Nevşehir", "Niğde", "Ordu", "Rize", "Sakarya", "Samsun", "Siirt", "Sinop", "Sivas", "Tekirdağ", "Tokat", "Trabzon", "Tunceli", "Şanlıurfa", "Uşak", "Van", "Yozgat", "Zonguldak", "Aksaray", "Bayburt", "Karaman", "Kırıkkale", "Batman", "Şırnak", "Bartın", "Ardahan", "Iğdır", "Yalova", "Karabük", "Kilis", "Osmaniye", "Düzce"
]

def temizle_metin(metin):
    if pd.isna(metin):
        return metin
    metin = str(metin).lower().strip()
    metin = re.sub(r'[^\w\s]', '', metin)
    return metin
# EŞİK DEĞİŞTİRİLEBİLİR
def en_yakin_eslesme(metin, secenekler, esik=70):
    if pd.isna(metin):
        return np.nan
    en_iyi_eslesme = process.extractOne(metin, secenekler)
    if en_iyi_eslesme[1] >= esik:
        return en_iyi_eslesme[0]
    return np.nan

def sehir_adi_temizle_ve_standartlastir(df, column_name):
    df[column_name] = df[column_name].apply(temizle_metin)
    df[column_name] = df[column_name].apply(lambda x: en_yakin_eslesme(x, sehirler))
    
    return df

train_X_raw = sehir_adi_temizle_ve_standartlastir(train_X_raw, 'Dogum Yeri')
train_X_raw = sehir_adi_temizle_ve_standartlastir(train_X_raw, 'Ikametgah Sehri')
train_X_raw['Dogum Yeri'] = train_X_raw['Dogum Yeri'].fillna('diger')
train_X_raw['Ikametgah Sehri'] = train_X_raw['Ikametgah Sehri'].fillna('diger')

frequency = train_X_raw['Dogum Yeri'].value_counts()
train_X_raw["encoded_dogum_yeri"] = train_X_raw['Dogum Yeri'].map(frequency)
train_X_raw.drop('Dogum Yeri', axis=1)

#train_X_raw['Ikametgah Sehri'].value_counts(dropna=False)

Dogum Yeri
İstanbul    11072
Ankara       4663
Diğer        3852
İzmir        2894
Bursa        1929
            ...  
Çankırı        98
Bayburt        79
Afyon          72
Tunceli        55
Muş             8
Name: count, Length: 82, dtype: int64

### Üniversite Adı(Frequency)

In [19]:
train_X_raw['Universite Adi'] = train_X_raw['Universite Adi'].str.upper()
train_X_raw['Universite Adi'] = train_X_raw['Universite Adi'].apply(lambda x: str(x).replace('İ', 'I').replace('Ü', 'U').replace('Ö', 'O'))
train_X_raw['Universite Adi'] = train_X_raw['Universite Adi'].fillna('DIGER')

frequency = train_X_raw['Universite Adi'].value_counts()
train_X_raw["encoded_universite_adi"] = train_X_raw['Universite Adi'].map(frequency)

train_X_raw['Universite Adi'].value_counts(dropna=False)
#train_X_raw["encoded_universite_adi"].value_counts(dropna=False)


Universite Adi
ISTANBUL UNIVERSITESI                                       2572
MARMARA UNIVERSITESI                                        2084
ISTANBUL TEKNIK UNIVERSITESI                                1954
YILDIZ TEKNIK UNIVERSITESI                                  1916
ORTA DOĞU TEKNIK UNIVERSITESI                               1658
                                                            ... 
BARD COLLEGE                                                   1
HARP AKADEMILERI (KARA, DENIZ,                                 1
ANKARA SOSYAL BILIMLER UNIVERSITESI KUZEY KIBRIS KAMPUSU       1
JANDARMA VE SAHIL GUVENLIK AKADEMISI(ASKERI)                   1
MILLI SAVUNMA UNIVERSITESI (ASKERÎ)                            1
Name: count, Length: 295, dtype: int64

### Üniversite Türü

In [34]:
train_X_raw["Universite Turu"] = train_X_raw["Universite Turu"].str.lower()
train_X_raw["Universite Turu"] = train_X_raw["Universite Turu"].fillna('diger')
#train_X_raw["Universite Turu"].value_counts(dropna=False)

Universite Turu
devlet    53262
özel      11608
diger       255
Name: count, dtype: int64

### 'Burslu ise Burs Yuzdesi', 'Burs Aliyor mu?(One Hot)

In [54]:
train_X_raw["Burs Aliyor mu?"] = train_X_raw["Burs Aliyor mu?"].str.lower()
train_X_raw["Burs Aliyor mu?"] = train_X_raw["Burs Aliyor mu?"].fillna("hayır")

train_X_raw.loc[train_X_raw['Burs Aliyor mu?'] == 'hayır', 'Burslu ise Burs Yuzdesi'] = np.nan
train_X_raw['Burslu ise Burs Yuzdesi'] = train_X_raw['Burslu ise Burs Yuzdesi'].fillna(int(0))

cut_labels = ['25altı', '25-49', '50-74', '75-99', '100']
cut_bins = [-1, 24.0, 49.0, 74.0, 99.0, 100.0]

train_X_raw['binned_burs_yuzde'] = pd.cut(train_X_raw['Burslu ise Burs Yuzdesi'], bins=cut_bins, labels=cut_labels)
train_X_raw.drop('Burslu ise Burs Yuzdesi', axis=1)

#train_X_raw['binned_burs_yuzde'].value_counts(dropna=False)

In [ ]:
encode_features = ['Ikametgah Sehri', 'Cinsiyet', 'Universite Turu', "Burs Aliyor mu?", "binned_burs_yuzde"]

encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

encoded_train = encoder.fit_transform(train_X_raw[encode_features])
categories = encoder.get_feature_names_out(encode_features)
encoded_df_train = pd.DataFrame(encoded_train, columns=categories)

encoded_test = encoder.transform(test_X_raw[encode_features])
encoded_df_test = pd.DataFrame(encoded_test, columns=categories)

train_X_raw = pd.concat([train_X_raw, encoded_df_train], axis=1)
test_X_raw = pd.concat([test_X_raw, encoded_df_test], axis=1)

train_X_raw.drop(encode_features, axis=1)

In [37]:
train_X_raw['Burslu ise Burs Yuzdesi'].value_counts(dropna=False).to_csv("a.csv")

In [32]:
train_raw.columns

Index(['Basvuru Yili', 'Degerlendirme Puani', 'Cinsiyet', 'Dogum Tarihi',
       'Dogum Yeri', 'Ikametgah Sehri', 'Universite Adi', 'Universite Turu',
       'Burslu ise Burs Yuzdesi', 'Burs Aliyor mu?', 'Bölüm',
       'Universite Kacinci Sinif', 'Universite Not Ortalamasi',
       'Daha Once Baska Bir Universiteden Mezun Olmus', 'Lise Adi',
       'Lise Adi Diger', 'Lise Sehir', 'Lise Turu', 'Lise Bolumu',
       'Lise Bolum Diger', 'Lise Mezuniyet Notu',
       'Baska Bir Kurumdan Burs Aliyor mu?', 'Burs Aldigi Baska Kurum',
       'Baska Kurumdan Aldigi Burs Miktari', 'Anne Egitim Durumu',
       'Anne Calisma Durumu', 'Anne Sektor', 'Baba Egitim Durumu',
       'Baba Calisma Durumu', 'Baba Sektor', 'Kardes Sayisi',
       'Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?',
       'Uye Oldugunuz Kulubun Ismi',
       'Profesyonel Bir Spor Daliyla Mesgul musunuz?',
       'Spor Dalindaki Rolunuz Nedir?', 'Aktif olarak bir STK üyesi misiniz?',
       'Hangi STK'nin Uyesisiniz?', 